In [1]:
import numpy as np
import matplotlib
matplotlib.rcParams['font.size'] = 9
matplotlib.rcParams.update({'figure.autolayout': True})
matplotlib.use("Agg")
from matplotlib import pyplot as plt
from matplotlib import cm

In [2]:
nodesizes = ["1", "2", "3-4", "5-8", "9-16", "17-32", "33-64", "65-128", "129-256", "257-512", "513-1024", "1025-2048", "2049-4096", "4097-8192"]
nodelabels = ["[1,1]", "[2,2]", "(2,4]", "(4,8]", "(8,16]", "(16,32]", "(32,64]", "(64,128]", "(128,256]", "(256,512]", "(512,1024]", "(1024,2048]", "(2048,4096]", "(4096,4290]"]
membins = [12.0, 24.0, 48.0, 96.0, 129.0]
memlabels = ['[0,12)','[12,24)','[24,48)','[48,96)','[96,128]']

In [3]:
def readSAFECSV(filename, membins, nodesizes):
    nsizes = len(nodesizes)
    nbins = len(membins)
    jobs = [[0 for i in range(nsizes)] for j in range(nbins)]
    use = [[0.0 for i in range(nsizes)] for j in range(nbins)]
    largejobs = 0
    largeuse = 0.0
    csvfile = open(filename, 'r')
    for line in csvfile:
        dataline = line.rstrip()
        tokens = dataline.split(",")
        inodes = nodesizes.index(tokens[0])
        imem = 0
        for mem in membins:
            if int(tokens[1]) < mem: break
            imem = imem + 1

        if imem >= nbins:
            largejobs += int(tokens[2])
            largeuse += float(tokens[3])
        else:
            jobs[imem][inodes] += int(tokens[2])
            use[imem][inodes] += float(tokens[3])

    csvfile.close()
    print largejobs, largeuse
    return jobs, use

In [4]:
def writeUsageHeatmap(imgfile, use, membins, nodesizes):
    nsizes = len(nodesizes)
    nbins = len(membins)
    xuse = np.array(use)
    totxuse = np.sum(xuse)
    fig, ax1 = plt.subplots()
    masked_array = np.ma.masked_where(xuse<0.0001, xuse)
    maxu = masked_array.max()
    cmap = cm.binary
    cmap.set_bad('w', 1.0)
    cax1 = ax1.imshow(masked_array, interpolation='nearest', cmap=cmap)
    plt.xticks(np.arange(nsizes), nodelabels, rotation='45')
    ax1.set_yticks(np.arange(nbins))
    ax1.set_yticklabels(memlabels)
    ax1.set_xlabel("Job size (nodes)")
    ax1.set_ylabel("Max. memory use (GiB/node)")
    for i in range(nbins):
        for j in range(nsizes):
            val =  xuse[i,j]/totxuse
            strval = "{:.1f}%".format(val*100)
            if xuse[i,j] > 0:
                if xuse[i,j]/maxu > 0.4:
                   ax1.text(j, i, strval, horizontalalignment='center', verticalalignment='center', color='white')
                else:
                   ax1.text(j, i, strval, horizontalalignment='center', verticalalignment='center')
            else:
                ax1.text(j, i, '-', horizontalalignment='center', verticalalignment='center')
    fig.savefig(imgfile, bbox_inches='tight', dpi=300)

In [5]:
jobs, use = readSAFECSV('mem_use.csv', membins, nodesizes)
writeUsageHeatmap('mem_use.png', use, membins, nodesizes)

2432 6081.966


/Users/aturner/anaconda/lib/python2.7/site-packages/matplotlib/figure.py:1744: UserWarning: This figure includes Axes that are not compatible with tight_layout, so its results might be incorrect.
  warnings.warn("This figure includes Axes that are not "


In [6]:
tot = np.sum(use)
[x/tot for x in np.sum(use,axis=0)]

[0.035278797878636858,
 0.018099608846912051,
 0.054227263937791537,
 0.16083745713696448,
 0.22887947676062204,
 0.25907936418705962,
 0.11189661712449384,
 0.081510254894621631,
 0.032905127732574403,
 0.01296753224015091,
 0.0043184992601728008,
 0.0,
 0.0,
 0.0]

In [7]:
[x/tot for x in np.sum(use,axis=1)]

[0.65902039156624048,
 0.20258586159314118,
 0.10028248214079806,
 0.035424584846822448,
 0.0026866798529980008]

In [8]:
appname = []
appuse = []
appfile = open('app_use.csv', 'r')
for line in appfile:
    line = line.rstrip()
    tokens = line.split(',')
    appname.append(tokens[0])
    appuse.append(tokens[4])
    
print appname
print appuse

['VASP', 'CP2K', 'CRYSTAL', 'ChemShell', 'CASTEP', 'AIMS', 'LAMMPS', 'NWChem', 'Unknown', 'GROMACS', 'Quantum Espresso', 'Python', 'NAMD', 'DL_POLY', 'CPMD', 'PIMAIM', 'GAMESS', 'SIESTA', 'ABINIT', 'ELK', 'AMBER']
['53.9', '16.8', '5', '4.8', '4.4', '3.5', '2.5', '2.2', '1.8', '1.3', '1.2', '0.6', '0.5', '0.5', '0.4', '0.3', '0.2', '0', '0', '0', '0']


In [9]:
import seaborn as sns
sns.set_style("white", {"font.family": "serif"})
sns.set_context("paper")
y_pos = np.arange(len(appname))
fig,ax = plt.subplots()
plt.barh(y_pos, appuse[::-1],align='center')
ax.set_yticks(y_pos)
ax.set_yticklabels(appname[::-1])
plt.ylim((0,len(appname)))
plt.xlabel("% Use")
sns.despine()
plt.savefig('app_use.png', bbox_inches='tight', dpi=300)